In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from machinedesign.autoencoder.interface import load
import keras.backend as K
from keras.models import Model
import numpy as np
from skimage.transform import resize, rescale
from skimage.io import imsave
from machinedesign.viz import grid_of_images_default

Using Theano backend.


In [102]:
digits = np.load('../../data/digits.npz')
digits = digits['X'] / 255.

In [2]:
model = load('../results/mnist')

In [139]:
#mnist
inp = model.layers[0].input
hid_layer = model.get_layer('winner_take_all_channel_1')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [11]:
data = np.load('../results/mnist/gen/generated.npz')
X = data['generated'][0:1000]

In [142]:
h = enc([X])
h = h.max(axis=(2, 3))
vals = h.max(axis=0)

In [106]:
H = np.zeros((hid_layer.input_shape[1],) + hid_layer.input_shape[1:])
H[np.arange(H.shape[0]), np.arange(H.shape[0]), 8, 8] = vals
x  = dec([H])
F = x
im = grid_of_images_default(x, normalize=False, shape=(16, 8))
imsave('features.png', im)

/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [143]:
d = digits[2:3]
h = enc([d])
hm = h.max(axis=(2, 3))
hm = hm[0]
f = []
f.append(d)
for i in range(len(hm)):
    if hm[i]:
        hh = h.copy()
        hh[:, 0:i] = 0
        hh[:, i+1:] = 0
        x = dec([hh])
        f.append(x)
f.append(d)
f = np.concatenate(f, axis=0)
print(f.shape)
im = grid_of_images_default(f, normalize=False, shape=(len(f), 1))
imsave('features_example.png', im)

(10, 1, 28, 28)


/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [56]:
dae = load('../results/mnist_dense2')
data = np.load('../../data/hwrt_thin.npz')
hwrt = data['X'] / 255.

In [64]:
x = hwrt[0:20] 
xr = x
for _ in range(1):
    xr = dae.predict(xr)
xl = np.array([x,xr])
xl = xl.transpose((1, 0, 2, 3, 4))
xl=xl.reshape((xl.shape[0]*xl.shape[1],) + xl.shape[2:])
im = grid_of_images_default(xl, shape=(len(x),2))
imsave('reconstructions_hwrt_dae.png', im)

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [77]:
x = hwrt[0:20] 
xr=x
xl=[x]
for _ in range(30):
    xr = dae.predict(xr)
    xl.append(xr)
xl = np.array(xl)
shape = xl.shape[0], xl.shape[1]
xl=xl.reshape((xl.shape[0]*xl.shape[1],) + xl.shape[2:])
im = grid_of_images_default(xl, shape=shape)
imsave('reconstructions_hwrt_dae_iter.png', im)

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
